## About iPython Notebooks ##

iPython Notebooks are interactive coding environments embedded in a webpage. You will be using iPython notebooks in this class. Make sure you fill in any place that says `# BEGIN CODE HERE #END CODE HERE`. After writing your code, you can run the cell by either pressing "SHIFT"+"ENTER" or by clicking on "Run" (denoted by a play symbol). Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All). 

 **What you need to remember:**

- Run your cells using SHIFT+ENTER (or "Run cell")
- Write code in the designated areas using Python 3 only
- Do not modify the code outside of the designated areas
- In some cases you will also need to explain the results. There will also be designated areas for that. 

Fill in your **NAME** and **AEM** below:

In [29]:
NAME = "Dimitrios Tikvinas"
AEM = "9998"

---

# Assignment 3 - Ensemble Methods #

Welcome to your third assignment. This exercise will test your understanding on Ensemble Methods.

In [30]:
# Always run this cell
import numpy as np
import pandas as pd

# USE THE FOLLOWING RANDOM STATE FOR YOUR CODE
RANDOM_STATE = 42

## Download the Dataset ##
Download the dataset using the following cell or from this [link](https://github.com/sakrifor/public/tree/master/machine_learning_course/EnsembleDataset) and put the files in the same folder as the .ipynb file. 
In this assignment you are going to work with a dataset originated from the [ImageCLEFmed: The Medical Task 2016](https://www.imageclef.org/2016/medical) and the **Compound figure detection** subtask. The goal of this subtask is to identify whether a figure is a compound figure (one image consists of more than one figure) or not. The train dataset consits of 4197 examples/figures and each figure has 4096 features which were extracted using a deep neural network. The *CLASS* column represents the class of each example where 1 is a compoung figure and 0 is not. 


In [31]:
import urllib.request
url_train = 'https://github.com/sakrifor/public/raw/master/machine_learning_course/EnsembleDataset/train_set.csv'
filename_train = 'train_set.csv'
urllib.request.urlretrieve(url_train, filename_train)
url_test = 'https://github.com/sakrifor/public/raw/master/machine_learning_course/EnsembleDataset/test_set_noclass.csv'
filename_test = 'test_set_noclass.csv'
urllib.request.urlretrieve(url_test, filename_test)

('test_set_noclass.csv', <http.client.HTTPMessage at 0x7f26efa248e0>)

In [32]:
# Run this cell to load the data
train_set = pd.read_csv("train_set.csv").sample(frac=1).reset_index(drop=True)
train_set.head()
X = train_set.drop(columns=['CLASS'])
y = train_set['CLASS'].values

In [33]:
!pip install -U imbalanced-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


The following code will reduce the number of instances, dealing with the small imbalance of the dataset, as well as reducing the size of the dataset!

In [34]:
from collections import Counter
from imblearn.under_sampling import NeighbourhoodCleaningRule, RandomUnderSampler

ncr = NeighbourhoodCleaningRule()
X_res, y_res = ncr.fit_resample(X, y)
rus = RandomUnderSampler(random_state=42)
X_res, y_res = rus.fit_resample(X_res, y_res)
print('Resampled dataset shape %s' % Counter(y_res))
X = X_res
y = y_res

Resampled dataset shape Counter({0: 1687, 1: 1687})


## 1.0 Testing different ensemble methods ##
In this part of the assignment you are asked to create and test different ensemble methods using the train_set.csv dataset. You should use **5-fold cross validation** for your tests and report the average f-measure weighted and balanced accuracy of your models. You can use [cross_validate](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html#sklearn.model_selection.cross_validate) and select both metrics to be measured during the evaluation. 

### !!! Use n_jobs=-1 where is posibble to use all the cores of a machine for running your tests ###

### 1.1 Voting ###
Create a voting classifier which uses two **simple** estimators/classifiers. Test both soft and hard voting and report the results. Consider as simple estimators the following:


*   Decision Trees
*   Linear Models
*   KNN Models  

In [35]:
### BEGIN SOLUTION

from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import model_selection

# USE RANDOM STATE! 
cls1 = DecisionTreeClassifier(random_state= RANDOM_STATE, max_depth=5) # Classifier #1 
cls2 = KNeighborsClassifier(n_neighbors=3)# Classifier #2
soft_vcls = VotingClassifier([('DecisionTree', cls1), ('KNeighbors', cls2)], voting = 'soft')# Voting Classifier
hard_vcls = VotingClassifier([('DecisionTree', cls1), ('KNeighbors', cls2)], voting = 'hard') # Voting Classifier

svlcs_scores = model_selection.cross_validate(soft_vcls, X, y, cv= 5, n_jobs= -1, scoring= ['f1_weighted', 'balanced_accuracy'])
s_avg_fmeasure = svlcs_scores['test_f1_weighted'].mean() # The average f-measure
s_avg_accuracy = svlcs_scores['test_balanced_accuracy'].mean() # The average accuracy

hvlcs_scores = model_selection.cross_validate(hard_vcls, X, y, cv= 5, n_jobs= -1, scoring= ['f1_weighted', 'balanced_accuracy'])
h_avg_fmeasure = hvlcs_scores['test_f1_weighted'].mean()  # The average f-measure
h_avg_accuracy = hvlcs_scores['test_balanced_accuracy'].mean() # The average accuracy

### END SOLUTION

print("Classifier:")
print(soft_vcls)
print("F1 Weighted-Score: {} & Balanced Accuracy: {}".format(round(s_avg_fmeasure,4), round(s_avg_accuracy,4)))

print("Classifier:")
print(hard_vcls)
print("F1 Weighted-Score: {} & Balanced Accuracy: {}".format(round(h_avg_fmeasure,4), round(h_avg_accuracy,4)))

Classifier:
VotingClassifier(estimators=[('DecisionTree',
                              DecisionTreeClassifier(max_depth=5,
                                                     random_state=42)),
                             ('KNeighbors',
                              KNeighborsClassifier(n_neighbors=3))],
                 voting='soft')
F1 Weighted-Score: 0.8566 & Balanced Accuracy: 0.8568
Classifier:
VotingClassifier(estimators=[('DecisionTree',
                              DecisionTreeClassifier(max_depth=5,
                                                     random_state=42)),
                             ('KNeighbors',
                              KNeighborsClassifier(n_neighbors=3))])
F1 Weighted-Score: 0.8078 & Balanced Accuracy: 0.8115


For both soft/hard voting classifiers the F1 weighted score should be above 0.74 and 0.79, respectively, and for balanced accuracy 0.74 and 0.80. Remember! This should be the average performance of each fold, as measured through cross-validation with 5 folds!

### 1.2 Randomization

You are asked to create three ensembles of decision trees where each one uses a different method for producing homogeneous ensembles. Compare them with a simple decision tree classifier and report your results in the dictionaries (dict) below using as key the given name of your classifier and as value the f1_weighted/balanced_accuracy score. The dictionaries should contain four different elements. Use the same cross-validation approach as before! 

In [36]:
### BEGIN SOLUTION
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
ens1 = RandomForestClassifier(n_estimators=200 ,random_state= RANDOM_STATE)
ens2 = ExtraTreesClassifier(random_state= RANDOM_STATE)
ens3 = GradientBoostingClassifier(random_state= RANDOM_STATE)
tree = DecisionTreeClassifier(random_state= RANDOM_STATE)

score1 = model_selection.cross_validate(ens1, X, y, cv= 5, n_jobs= -1, scoring= ['f1_weighted', 'balanced_accuracy'])
score2 = model_selection.cross_validate(ens2, X, y, cv= 5, n_jobs= -1, scoring= ['f1_weighted', 'balanced_accuracy'])
score3 = model_selection.cross_validate(ens3, X, y, cv= 5, n_jobs= -1, scoring= ['f1_weighted', 'balanced_accuracy'])
score4 = model_selection.cross_validate(tree, X, y, cv= 5, n_jobs= -1, scoring= ['f1_weighted', 'balanced_accuracy'])

f_measures = dict({"Random Forest": score1['test_f1_weighted'].mean() , "Extra Trees": score2['test_f1_weighted'].mean(),
                   "Grad Boost": score3['test_f1_weighted'].mean(), "Decision Tree": score4['test_f1_weighted'].mean() })

accuracies = dict({"Random Forest": score1['test_balanced_accuracy'].mean() , "Extra Trees": score2['test_balanced_accuracy'].mean(),
                   "Grad Boost": score3['test_balanced_accuracy'].mean(), "Decision Tree": score4['test_balanced_accuracy'].mean() })
# Example f_measures = {'Simple Decision':0.8551, 'Ensemble with random ...': 0.92, ...}


### END SOLUTION

print(ens1)
print(ens2)
print(ens3)
print(tree)
for name,score in f_measures.items():
    print("Classifier: {} -  F1 Weighted: {}".format(name,round(score,4)))
for name,score in accuracies.items():
    print("Classifier: {} -  BalancedAccuracy: {}".format(name,round(score,4)))

RandomForestClassifier(n_estimators=200, random_state=42)
ExtraTreesClassifier(random_state=42)
GradientBoostingClassifier(random_state=42)
DecisionTreeClassifier(random_state=42)
Classifier: Random Forest -  F1 Weighted: 0.8642
Classifier: Extra Trees -  F1 Weighted: 0.8621
Classifier: Grad Boost -  F1 Weighted: 0.8592
Classifier: Decision Tree -  F1 Weighted: 0.7459
Classifier: Random Forest -  BalancedAccuracy: 0.8642
Classifier: Extra Trees -  BalancedAccuracy: 0.8622
Classifier: Grad Boost -  BalancedAccuracy: 0.8592
Classifier: Decision Tree -  BalancedAccuracy: 0.746


### 1.3 Question

Increasing the number of estimators in a bagging classifier can drastically increase the training time of a classifier. Is there any solution to this problem? Can the same solution be applied to boosting classifiers?

YOUR ANSWER HERE

As the number of estimators increases, the training time scales linearly because each estimator needs to be trained independently, as the way the bagging classifier works, meaning that it involves creating multiple subsets of the original dataset and training a separate classifier on each subset.

 One way to solve this problem is to parallelize the training process, an approach which doesn't disturb the classifier's functionality because, as we said earlier, each estimator is trained independently. 
 
 No, we cannot use the same solution to boosting classifiers, due to their approach of typically having a sequential training process, where each subsequent estimator focuses on the samples that were misclassified by the previous ones. 

## 2.0 Creating the best classifier ##
In the second part of this assignment, we will try to train the best classifier, as well as to evaluate it using stratified cross valdiation.

### 2.1 Good Performing Ensemble

In this part of the assignment you are asked to train a good performing ensemble, that is able to be used in a production environment! Describe the process you followed to achieve this result. How did you choose your classifier and your parameters and why. Report the f-measure (weighted) & balanced accuracy, using 10-fold stratified cross validation, of your final classifier. Can you achieve a balanced accuracy over 88%, while keeping the training time low? (Tip 1: You can even use a model from the previous parts, but you are advised to test additional configurations, and ensemble architectures, Tip 2: If you try a lot of models/ensembles/configurations or even grid searches, in your answer leave only the classifier you selected as the best!)

In [37]:
### BEGIN SOLUTION
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import f1_score, balanced_accuracy_score
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
    
# Example f_measures = {'Simple Decision':0.8551, 'Ensemble with random ...': 0.92, ...}
cls1 = MLPClassifier(random_state= RANDOM_STATE)
cls2 = SVC(random_state=RANDOM_STATE, degree = 4, C = 10) 
cls = [('rf', cls1), ('svc', cls2)]

best_cls = VotingClassifier(cls, voting = 'hard', n_jobs = -1)

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)


for train_index, val_index in skf.split(X, y):
    X_train_fold, X_val_fold = X.iloc[train_index], X.iloc[val_index]
    y_train_fold, y_val_fold = y[train_index], y[val_index]

    best_cls.fit(X_train_fold, y_train_fold)

    y_val_pred = best_cls.predict(X_val_fold)
    score_balanced_accuracy = balanced_accuracy_score(y_val_fold, y_val_pred)
    score_f1 = f1_score(y_val_fold, y_val_pred)


best_fmeasure = score_f1.mean()
best_accuracy = score_balanced_accuracy.mean()
## END SOLUTION

print("Classifier:")
print(best_cls)
print("F1 Weighted-Score: {} & Balanced Accuracy: {}".format(best_fmeasure, best_accuracy))

Classifier:
VotingClassifier(estimators=[('rf', MLPClassifier(random_state=42)),
                             ('svc', SVC(C=10, degree=4, random_state=42))],
                 n_jobs=-1)
F1 Weighted-Score: 0.8957055214723926 & Balanced Accuracy: 0.8992145674837982


LEAVE HERE ANY COMMENTS ABOUT YOUR CLASSIFIER

After picking this final configuration for my classifier, i runned a grid search in order to find the most optimized hyperparameters. That's it pretty much.

### 2.2 Question
 What other ensemble architectures you tried, and why you did not choose them as your final classifier?

YOUR ANSWER HERE

From the previous sections we concluded that the *RandomForestClassifier* produced the model with the highest accuracy on the training data, so we will work on with it. In our pursuit of the best classifier, we will also use SVM classifiers and Neural Networks ones (MLP) as two of the most efficient and powerful classifiers.

Below we can see the training duration of each designed classifier, the balanced accuracy it achieved after 10-fold stratified cross validation and its structure

(10 secs) -> 85.45% Ada Boost -> RandomForest

(30 secs) -> 86.63% Voting, hard -> Random Forest, SVC

(50 secs) -> 87.83% Bagging -> RandomForest

(50 secs) -> 89.02% Voting, hard -> MLP, SVM

(90 secs) -> 89.30% Bagging -> SVC

(140 secs) -> 88.83% Bagging -> MLP

(220 secs) -> 88.13% Stacking -> RandomForest, SVC

Through *Hard Voting -> RandomForest, SVC* we achieved the highest accuracy in the least amount of time, being above the declared specification of 88%.

### 2.3 Setup the Final Classifier
Finally, in this last cell, set the cls variable to either the best model as occured by the stratified cross_validation, or choose to retrain your classifier in the whole dataset (X, y). There is no correct answer, but try to explain your choice. Then, save your model using pickle and upload it with your submission to e-learning!

In [38]:
import pickle

### BEGIN SOLUTION
cls = best_cls

# save with pickle
file_name = "my_best_classifier_9998.pkl"
pickle.dump(cls, open(file_name, "wb"))
### END SOLUTION


# load
cls = pickle.load(open(file_name, "rb"))

test_set = pd.read_csv("test_set_noclass.csv")
predictions = cls.predict(test_set)

# We are going to run the following code
if False:
  from sklearn.metrics import f1_score, balanced_accuracy_score
  final_test_set = pd.read_csv('test_set.csv')
  ground_truth = final_test_set['CLASS']
  print("Balanced Accuracy: {}".format(balanced_accuracy_score(predictions, ground_truth)))
  print("F1 Weighted-Score: {}".format(f1_score(predictions, ground_truth, average='weighted')))

Both metrics should aim above 82%! This is going to be tested by us! Make sure your cross validation or your retrained model achieves high balanced accuracy and f1_score (based on 2.1) (more than 88%) as it should achieve at least 82% in our unknown test set!


Please provide your feedback regarding this project! Did you enjoy it? 

In [39]:
# YOUR ANSWER HERE
#It was quite creative, trying to figuring out which classifier's configuration to choose. It made me feel like I was doing a real-world application.